In [26]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
file_path = "Space_Corrected.csv.xls"
try:
    df = pd.read_excel(file_path)
    print("Loaded using read_excel()")
except Exception as e:
    print("read_excel failed, trying read_csv instead:", e)
    df = pd.read_csv(file_path)
    print("Loaded using read_csv()")

df.head()

read_excel failed, trying read_csv instead: Excel file format cannot be determined, you must specify an engine manually.
Loaded using read_csv()


,Unnamed: 0.1,Unnamed: 0,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [28]:
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
df.info()

Shape: (4324, 9)

Columns:
['Unnamed: 0.1', 'Unnamed: 0', 'Company Name', 'Location', 'Datum', 'Detail', 'Status Rocket', ' Rocket', 'Status Mission']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    4324 non-null   int64 
 1   Unnamed: 0      4324 non-null   int64 
 2   Company Name    4324 non-null   object
 3   Location        4324 non-null   object
 4   Datum           4324 non-null   object
 5   Detail          4324 non-null   object
 6   Status Rocket   4324 non-null   object
 7    Rocket         964 non-null    object
 8   Status Mission  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


In [29]:
df.sample(5)

for col in df.columns[:5]:
    print(f"\nColumn: {col}")
    print(df[col].unique()[:10])


Column: Unnamed: 0.1
[0 1 2 3 4 5 6 7 8 9]

Column: Unnamed: 0
[0 1 2 3 4 5 6 7 8 9]

Column: Company Name
['SpaceX' 'CASC' 'Roscosmos' 'ULA' 'JAXA' 'Northrop' 'ExPace' 'IAI'
 'Rocket Lab' 'Virgin Orbit']

Column: Location
['LC-39A, Kennedy Space Center, Florida, USA'
 'Site 9401 (SLS-2), Jiuquan Satellite Launch Center, China'
 'Pad A, Boca Chica, Texas, USA'
 'Site 200/39, Baikonur Cosmodrome, Kazakhstan'
 'SLC-41, Cape Canaveral AFS, Florida, USA'
 'LC-9, Taiyuan Satellite Launch Center, China'
 'Site 31/6, Baikonur Cosmodrome, Kazakhstan'
 'LC-101, Wenchang Satellite Launch Center, China'
 'SLC-40, Cape Canaveral AFS, Florida, USA'
 'LA-Y1, Tanegashima Space Center, Japan']

Column: Datum
['Fri Aug 07, 2020 05:12 UTC' 'Thu Aug 06, 2020 04:01 UTC'
 'Tue Aug 04, 2020 23:57 UTC' 'Thu Jul 30, 2020 21:25 UTC'
 'Thu Jul 30, 2020 11:50 UTC' 'Sat Jul 25, 2020 03:13 UTC'
 'Thu Jul 23, 2020 14:26 UTC' 'Thu Jul 23, 2020 04:41 UTC'
 'Mon Jul 20, 2020 21:30 UTC' 'Sun Jul 19, 2020 21:58 UTC']


In [30]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,4324.0,2161.5,1248.375611,0.0,1080.75,2161.5,3242.25,4323.0
Unnamed: 0,4324.0,2161.5,1248.375611,0.0,1080.75,2161.5,3242.25,4323.0


In [31]:
df.isnull().sum()

Unnamed: 0.1         0
Unnamed: 0           0
Company Name         0
Location             0
Datum                0
Detail               0
Status Rocket        0
 Rocket           3360
Status Mission       0
dtype: int64

In [32]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

df = df.rename(columns={' Rocket': 'Rocket', 'Datum': 'Date'})

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()

df['Country'] = df['Location'].apply(lambda x: x.split(',')[-1].strip() if isinstance(x, str) else None)

df['Status Mission'] = df['Status Mission'].str.strip().str.title()
df['Status Rocket'] = df['Status Rocket'].str.strip().str.title()

df = df.dropna(subset=['Date'])

df.head()

,Company Name,Location,Date,Detail,Status Rocket,Rocket,Status Mission,Year,Month,Country
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,Statusactive,50.0,Success,2020.0,August,USA
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,Statusactive,29.75,Success,2020.0,August,China
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,Statusactive,NaN,Success,2020.0,August,USA
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,Statusactive,65.0,Success,2020.0,July,Kazakhstan
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,Statusactive,145.0,Success,2020.0,July,USA


In [33]:
print("Shape after cleaning:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample rows:")
display(df.sample(5))

print("\nUnique mission statuses:", df['Status Mission'].unique())
print("\nUnique rocket statuses:", df['Status Rocket'].unique())

Shape after cleaning: (4198, 10)

Columns: ['Company Name', 'Location', 'Date', 'Detail', 'Status Rocket', 'Rocket', 'Status Mission', 'Year', 'Month', 'Country']

Sample rows:


,Company Name,Location,Date,Detail,Status Rocket,Rocket,Status Mission,Year,Month,Country
1839,RVSN USSR,"Site 32/2, Plesetsk Cosmodrome, Russia",1990-02-28 00:55:00+00:00,Tsyklon-3 | Okean 2,Statusretired,NaN,Success,1990.0,February,Russia
2644,General Dynamics,"SLC-3W, Vandenberg AFB, California, USA",1977-12-08 17:45:00+00:00,Atlas-E/F MSD | NOSS-2,Statusretired,NaN,Success,1977.0,December,USA
700,ULA,"SLC-6, Vandenberg AFB, California, USA",2011-01-20 21:10:00+00:00,Delta IV Heavy | NROL-49,Statusactive,350.0,Success,2011.0,January,USA
1443,NASA,"LC-39B, Kennedy Space Center, Florida, USA",1996-05-19 10:30:00+00:00,Space Shuttle Endeavour | STS-77,Statusretired,450.0,Success,1996.0,May,USA
202,Landspace,"Site 95, Jiuquan Satellite Launch Center, China",2018-10-27 08:00:00+00:00,ZhuQue-1 | CCTV Future-1,Statusretired,NaN,Failure,2018.0,October,China



Unique mission statuses: ['Success' 'Failure' 'Partial Failure' 'Prelaunch Failure']

Unique rocket statuses: ['Statusactive' 'Statusretired']


In [34]:
df.to_csv("cleaned_space_missions.csv", index=False)
print("Saved cleaned dataset as cleaned_space_missions.csv")

Saved cleaned dataset as cleaned_space_missions.csv


EDA

In [35]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

df = pd.read_csv("cleaned_space_missions.csv")

In [36]:
df.shape

(4198, 10)

In [37]:
df.dtypes

Company Name       object
Location           object
Date               object
Detail             object
Status Rocket      object
Rocket             object
Status Mission     object
Year              float64
Month              object
Country            object
dtype: object

In [38]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('-', '_')
df.head()

,Company_Name,Location,Date,Detail,Status_Rocket,Rocket,Status_Mission,Year,Month,Country
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,Statusactive,50.0,Success,2020.0,August,USA
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,Statusactive,29.75,Success,2020.0,August,China
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,Statusactive,NaN,Success,2020.0,August,USA
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,Statusactive,65.0,Success,2020.0,July,Kazakhstan
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,Statusactive,145.0,Success,2020.0,July,USA


In [39]:
print("Data shape:", df.shape)
print("\nColumn names:\n", df.columns.tolist())

Data shape: (4198, 10)

Column names:
 ['Company_Name', 'Location', 'Date', 'Detail', 'Status_Rocket', 'Rocket', 'Status_Mission', 'Year', 'Month', 'Country']


In [40]:
print("Year range:", df['Year'].min(), "-", df['Year'].max())
print("Unique countries:", df['Country'].nunique())
print("Unique companies:", df['Company_Name'].nunique())
print("\nMission status distribution:\n", df['Status_Mission'].value_counts())
print("\nRocket status distribution:\n", df['Status_Rocket'].value_counts())

Year range: 1957.0 - 2020.0
Unique countries: 21
Unique companies: 55

Mission status distribution:
 Status_Mission
Success              3794
Failure               300
Partial Failure       102
Prelaunch Failure       2
Name: count, dtype: int64

Rocket status distribution:
 Status_Rocket
Statusretired    3432
Statusactive      766
Name: count, dtype: int64


In [41]:
df[['Year', 'Country', 'Company_Name', 'Status_Mission', 'Status_Rocket']].head(10)

,Year,Country,Company_Name,Status_Mission,Status_Rocket
0,2020.0,USA,SpaceX,Success,Statusactive
1,2020.0,China,CASC,Success,Statusactive
2,2020.0,USA,SpaceX,Success,Statusactive
3,2020.0,Kazakhstan,Roscosmos,Success,Statusactive
4,2020.0,USA,ULA,Success,Statusactive
5,2020.0,China,CASC,Success,Statusactive
6,2020.0,Kazakhstan,Roscosmos,Success,Statusactive
7,2020.0,China,CASC,Success,Statusactive
8,2020.0,USA,SpaceX,Success,Statusactive
9,2020.0,Japan,JAXA,Success,Statusactive


In [42]:
launches_per_year = df.groupby('Year').size().reset_index(name='Launches')
launches_per_year.head()

,Year,Launches
0,1957.0,3
1,1958.0,22
2,1959.0,20
3,1960.0,38
4,1961.0,52


In [43]:
import plotly.express as px

fig1 = px.line(
    launches_per_year,
    x='Year',
    y='Launches',
    title='Number of Space Launches Per Year',
    markers=True
)
fig1.update_layout(
    xaxis=dict(rangeslider=dict(visible=True)),
    hovermode='x unified'
)
fig1.show()

In [44]:
launches_by_country = df['Country'].value_counts().reset_index(name='Total_Launches')
launches_by_country = launches_by_country.rename(columns={'index':'Country'})
launches_by_country.head(10)

,Country,Total_Launches
0,Russia,1351
1,USA,1321
2,Kazakhstan,679
3,France,303
4,China,264
5,Japan,122
6,India,75
7,Pacific Ocean,32
8,New Zealand,13
9,Kenya,9


In [45]:
import plotly.express as px

fig2 = px.bar(
    launches_by_country.head(15),
    x='Country',
    y='Total_Launches',
    title='Top 15 Countries by Total Space Launches'
)
fig2.update_layout(xaxis_title='Country', yaxis_title='Total Launches', xaxis_tickangle=-45)
fig2.show()

In [46]:
launches_by_company = (
    df['Company_Name']
    .value_counts()
    .reset_index(name='Total_Launches')
    .rename(columns={'index': 'Company_Name'})
)

In [47]:
fig3 = px.bar(
    launches_by_company.head(20),
    x='Total_Launches',
    y='Company_Name',
    orientation='h',
    title='Top 20 Organizations by Number of Launches',
    text='Total_Launches'
)

fig3.update_traces(textposition='outside')
fig3.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    height=900,
    margin=dict(l=240, r=50, t=80, b=50),
    xaxis_title='Total Launches',
    yaxis_title='Organization',
    font=dict(size=12)
)
fig3.show()

In [48]:
mission_outcomes = df.groupby(['Year', 'Status_Mission']).size().reset_index(name='Count')

fig4 = px.bar(
    mission_outcomes,
    x='Year',
    y='Count',
    color='Status_Mission',
    barmode='stack',
    title='Mission Outcomes Over Time'
)
fig4.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Missions',
    legend_title='Mission Status'
)
fig4.show()

In [49]:
rocket_status = df['Status_Rocket'].value_counts().reset_index(name='Count')
rocket_status = rocket_status.rename(columns={'index': 'Status_Rocket'})

fig5 = px.pie(
    rocket_status,
    names='Status_Rocket',
    values='Count',
    title='Rocket Status Distribution (Active vs Retired)',
    hole=0.3
)
fig5.update_traces(textinfo='percent+label')
fig5.show()

In [50]:
import os
os.makedirs('figures', exist_ok=True)
fig1.write_image("figures/launches_per_year.png")
fig2.write_image("figures/top_countries.png")
fig3.write_image("figures/top_organizations.png")
fig4.write_image("figures/mission_outcomes.png")
fig5.write_image("figures/rocket_status.png")